In [ ]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import csv
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn import svm
from pandas.tseries.offsets import DateOffset
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

%matplotlib inline

In [ ]:
# Load the environment variables by calling the load_dotenv function
load_dotenv()

In [ ]:
# Set Alpaca API key and secret by calling the os.getenv function and referencing the environment variable names
# Set each environment variable to a notebook variable of the same name
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Check the values were imported correctly by evaluating the type of each
display(type(alpaca_api_key))
display(type(alpaca_secret_key))


In [ ]:
# Create your Alpaca API REST object by calling Alpaca's tradeapi.REST function
# Set the parameters to your alpaca_api_key, alpaca_secret_key and api_version="v2"
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")


In [ ]:
# Format the current date as ISO format
today = pd.Timestamp("2021-09-28", tz="America/New_York").isoformat()

In [ ]:
# Set the tickers for the stocks you have chosen
# We are selecting the following 15 (note to group that 14 currently work, but added in VOO)
tickers = ["TSM", "QCOM", "VALE", "AMD", "BHP", "RIO", "FCX", "INTC", "MSFT", "DDD", "NVDA", "TSLA", "AMAT", "F", "VOO"]

In [ ]:
# Set timeframe to one day ('1D') for the Alpaca API
timeframe = "1D"

In [ ]:
# Get current closing prices for above stocks
df_portfolio = alpaca.get_barset(
    tickers,
    timeframe,
    start = today,
    end = today,
    limit = 500
).df

# Display sample data
# Auto formats in alphabetical order
df_portfolio

In [ ]:
# The farthest Alpaca will go back is 4 months for one pull
# Format start and end dates as ISO format for one year period
start = pd.Timestamp("2021-05-28", tz="America/New_York").isoformat()
end = pd.Timestamp("2021-09-28", tz="America/New_York").isoformat()

In [ ]:
# Get closing prices for chosen stocks from start to end date of choice
df_portfolio_months = alpaca.get_barset(
    tickers,
    timeframe,
    start = start,
    end = end,
    limit = 1000
).df

# Display sample data
df_portfolio_months.head(10)

In [ ]:
# Clean the data and drop all columns except for closing
df_portfolio_months_clean = df_portfolio_months.drop(['open', 'low', 'high'], axis=1, level=1)

# Display sample data
df_portfolio_months_clean.head(10)

In [ ]:
# The farthest Alpaca will go back is 4 months for one pull, so we repeat process below
# Format start and end dates as ISO format for four month period
# Use the start date above as the end date below
start_2 = pd.Timestamp("2021-01-01", tz="America/New_York").isoformat()
end_2 = pd.Timestamp("2021-05-27", tz="America/New_York").isoformat()

In [ ]:
# Get closing prices for chosen stocks from start to end date of choice
df_portfolio_months_2 = alpaca.get_barset(
    tickers,
    timeframe,
    start = start_2,
    end = end_2,
    limit = 1000
).df

# Clean the data and drop all columns except for closing
df_portfolio_months_2_clean = df_portfolio_months_2.drop(['open', 'low', 'high'], axis=1, level=1)

# Display sample data
df_portfolio_months_2_clean.head(10)

In [ ]:
# The farthest Alpaca will go back is 4 months for one pull, so we repeat process below
# Format start and end dates as ISO format for four month period
# Use the start date above as the end date below

start_3 = pd.Timestamp("2020-09-28", tz="America/New_York").isoformat()
end_3 = pd.Timestamp("2021-01-01", tz="America/New_York").isoformat()

df_portfolio_months_3 = alpaca.get_barset(
    tickers,
    timeframe,
    start = start_3,
    end = end_3,
    limit = 1000
).df

# Clean the data and drop all columns except for closing
df_portfolio_months_3_clean = df_portfolio_months_3.drop(['open', 'low', 'high'], axis=1, level=1)

# Display sample data
df_portfolio_months_3_clean.head(10)

In [ ]:
#Combine all the cleaned data into one table by using concat
merged_portfolio = pd.concat([df_portfolio_months_3_clean, df_portfolio_months_2_clean, df_portfolio_months_clean])

# Display the head and tail of the newly merged portfolio
display(merged_portfolio.head())
display(merged_portfolio.tail())

In [ ]:
# Assign save path to variable
save_path = Path('./cleaned_stock_data.csv')

# Send to CSV file if file does not already exist
if not save_path.is_file():
    merged_portfolio.to_csv(save_path)
else:
    # alright, proceed...
    print('csv output not resaved.')

In [ ]:
# Graph your stocks closing prices over the course of the last year by using the data you just merged into one portfolio
# We are graphing the data to get a visualization over the stock closing price performance over the last year

# First remove the volume column from the portfolio to just plot the closing prices
merged_portfolio_close_dates = merged_portfolio.drop(['volume'], axis=1, level=1)

merged_portfolio_close_dates.plot()
plt.title('Full Calendar Year of Closing Prices for Chosen Stocks')
plt.ylabel('Closing Prices')
plt.xlabel('Date')
plt.legend(loc="lower right", bbox_to_anchor=(1.4, -0.19), ncol=1)

In [ ]:
# Graph your stocks daily volume over the course of the last year by using the data you just merged into one portfolio
# We are graphing the data to get a visualization over the changes in volume over the last year

# First remove the close column from the portfolio to just plot the volume
merged_portfolio_volumes = merged_portfolio.drop(['close'], axis=1, level=1)

merged_portfolio_volumes.plot()
plt.title('Full Calendar Year of Volume for Chosen Stocks')
plt.ylabel('Volume')
plt.xlabel('Date')
plt.legend(loc="lower right", bbox_to_anchor=(1.4, -0.19), ncol=1)

In [ ]:
# Create a Lagged Volume column that shifts the volume of each stock back by one day

# Since there are multiple columns with the name volume, we must create a loop 
for widget in merged_portfolio.columns.levels[0]:
    merged_portfolio.loc[:,(widget, 'lagged volume')] = merged_portfolio[(widget, 'volume')].shift(1)

# Then we sort the columns to show within each stock
merged_portfolio = merged_portfolio.sort_index(axis=1, level=0)

#Drop NA values
merged_portfolio.dropna(inplace=True)

#Display portfolio 
merged_portfolio.head(10)

In [ ]:
# Create a Daily Stock Return column that will calculate the daily percent change of the closing stock prices

# Since there are multiple columns with the name close, we must create a loop 

for stock in merged_portfolio.columns.levels[0]:
    merged_portfolio.loc[:,(stock, 'daily return')] = merged_portfolio[(stock, 'close')].pct_change()

# Then we sort the columns to show within each stock
merged_portfolio = merged_portfolio.sort_index(axis=1, level=0)

#Display first and last five rows of the portfolio 
display(merged_portfolio.head())
display(merged_portfolio.tail())

In [ ]:
# Create a Stock Volatility column that will calculate the standard deviation of the closing stock prices

# Since there are multiple columns with the name close, we must create a loop 

for volatility in merged_portfolio.columns.levels[0]:
    merged_portfolio.loc[:,(volatility, 'stock volatility')] = merged_portfolio[(volatility, 'close')].pct_change().rolling(window=200).std()

# Then we sort the columns to show within each stock
merged_portfolio = merged_portfolio.sort_index(axis=1, level=0)

#Display portfolio 
merged_portfolio.tail(10)

In [ ]:
# Graph your stock volatility over the course of the last year by using the data you calculated above
# We are graphing the data to get a visualization over the stock volatility over the last year

# First remove the close, volume, lagged volume, and daily return columns from the portfolio to just plot the volume

merged_portfolio_stock_volatility = merged_portfolio.drop(['close', 'volume', 'lagged volume', 'daily return'], axis=1, level=1)

merged_portfolio_stock_volatility.plot()
plt.title('Stock Volatility for Chosen Stocks')
plt.ylabel('Volatility')
plt.xlabel('Date')
plt.legend(loc="lower right", bbox_to_anchor=(1.53, -0.19), ncol=1)

In [ ]:
# Creating a correlation table of the daily stock return, lagged volume, and stock volatility

merged_portfolio_corr = merged_portfolio.drop(['close', 'volume'], axis=1, level=1)

merged_portfolio_corr.corr()

In [ ]:
# Original idea to calculate correlation 

# merged_portfolio_TSM_corr = merged_portfolio.drop(["QCOM", "VALE", "AMD", "BHP", "RIO", "FCX", "INTC", "MSFT", "DDD", "NVDA", "TSLA", "AMAT", "F", "VOO"], axis=1, level=0)

# merged_portfolio_TSM_corr.corr()

In [ ]:
# Below we start running Bollinger Bands to determine return percentage

In [ ]:
merged_portfolio_daily_returns = merged_portfolio.drop(['close', 'volume', 'lagged volume', 'stock volatility'], axis=1, level=1)

merged_portfolio_daily_returns.dropna(inplace=True)

merged_portfolio_daily_returns.head(10)

In [ ]:
# Original strategy to get the covariance of each stock in portfolio

# Below calculates the covariance of TSM to the VOO

merged_portfolio_TSM_cov = merged_portfolio_daily_returns['TSM']['daily return'].cov(merged_portfolio_daily_returns['VOO']['daily return'])

merged_portfolio_TSM_cov



In [ ]:
# Creating a loop to get all the stock covariances to the VOO at once in the order below

stock_ticker_list = ["TSM", "QCOM", "VALE", "AMD", "BHP", "RIO", "FCX", "INTC", "MSFT", "DDD", "NVDA", "TSLA", "AMAT", "F"]

stock_covariance_list = []

for stock in stock_ticker_list:
    stockcov = merged_portfolio_daily_returns[stock]['daily return'].cov(merged_portfolio_daily_returns['VOO']['daily return'])
    stock_covariance_list+=[stockcov]

print(stock_covariance_list)
   

In [ ]:
# Create the variance calculation of all the daily returns vs. VOO

VOO_variance = merged_portfolio_daily_returns['VOO']['daily return'].var()

print(VOO_variance)



In [ ]:
# trying to create a loop that divided the covariance of each stock by the variance of VOO, both of which have been calculated above already

stock_beta_list = []

for covariance in stock_covariance_list:
    stockbeta = covariance / VOO_variance
    stock_beta_list += [stockbeta]

print(stock_beta_list)
    

In [ ]:
# Remove VOO from the stock_beta_list
# Assign the VOO beta to a variable
voo_beta = stock_beta_list.pop()

# Print the beta of VOO
print(voo_beta)

In [ ]:
# Assign the stock_beta_list without voo to stock_beta_list_wo_voo
stock_beta_list_wo_voo = stock_beta_list

# Print stock_beta_list_wo_voo
print(stock_beta_list_wo_voo)


In [ ]:
# If we used even weights for all stocks in the portfolio
# The portfolio beta would be the sum of the betas for all the stocks over the number of stocks in the portfolio
# Calculating the portfolio beta
stock_beta_list_wo_voo_sum = sum(stock_beta_list_wo_voo)
stock_beta_list_wo_voo_count = len(stock_beta_list_wo_voo)
stock_beta_list_wo_voo_beta = stock_beta_list_wo_voo_sum / stock_beta_list_wo_voo_count

# Print the portfolio beta given even weights for all stocks
print(stock_beta_list_wo_voo_beta)

In [ ]:
# Create a new DataFrame trading_df using the merged_portfolio DataFrame
trading_df = merged_portfolio

# Display the first five rows of the DataFrame
trading_df.head()

In [ ]:
# Display Column headers for the trading_df
display(list(trading_df.columns.values))

In [ ]:
# Drop VOO from the trading_df DataFRame
trading_df = trading_df.drop(['VOO'], axis=1)

# View the Updated DataFrame Columns
display(list(trading_df.columns.values))

In [ ]:
# View the Updated DataFrame
display(trading_df.head())
display(trading_df.tail())

In [ ]:
# Define a window size of 4
short_window = 4

# Define a window size of 100
long_window = 100

# Re-define stock_ticker_list to ensure integrity
stock_ticker_list = ["AMAT", "AMD", "BHP", "DDD", "F", "FCX", "INTC", "MSFT", "NVDA", "QCOM", "RIO", "TSLA", "TSM", "VALE"]

# Loop through the trading_df to create the sma_fast and sma_slow columns 
for stock in stock_ticker_list:
    # Create an SMA that uses short_window, and assign it to a new column named “sma_fast”
    trading_df.loc[:,(stock, 'sma_fast')] = trading_df[(stock, 'close')].rolling(window=short_window).mean()
    # Create an SMA that uses long_window, and assign it to a new column named “sma_slow”
    trading_df.loc[:,(stock, 'sma_slow')] = trading_df[(stock, 'close')].rolling(window=long_window).mean()
    
# Then we sort the columns to show within each stock
trading_df = trading_df.sort_index(axis=1, level=0)
    
# Display the first and last five rows of the trading_df
display(trading_df.head())
display(trading_df.tail())


In [ ]:
# Drop the NaNs using dropna()
trading_df.dropna(axis=0, inplace=True)

# Display the first and last five rows
display(trading_df.head())
display(trading_df.tail())

In [ ]:
# Create X DataFrame
X = pd.DataFrame(columns = trading_df.columns)

# Re-define stock_ticker_list to ensure integrity
stock_ticker_list = ["AMAT", "AMD", "BHP", "DDD", "F", "FCX", "INTC", "MSFT", "NVDA", "QCOM", "RIO", "TSLA", "TSM", "VALE"]

# Assign a copy of the `sma_fast` and `sma_slow` columns to the new DataFrame called `X`
# Loop through the trading_df
for stock in stock_ticker_list:
    X.loc[:,(stock, 'sma_fast')] = trading_df[(stock, 'sma_fast')].shift().dropna().copy()
    X.loc[:,(stock, 'sma_slow')] = trading_df[(stock, 'sma_slow')].shift().dropna().copy()

# Drop columns other than sma_fast and sma_slow
X = X.drop(['close', 'daily return', 'lagged volume', 'stock volatility', 'volume'], axis='columns', level=1)

# Display the first and last five rows of the X DataFrame
display(X.head())
display(X.tail())


In [ ]:
# Re-define stock_ticker_list to ensure integrity
stock_ticker_list = ["AMAT", "AMD", "BHP", "DDD", "F", "FCX", "INTC", "MSFT", "NVDA", "QCOM", "RIO", "TSLA", "TSM", "VALE"]

# Create a new column in the `trading_df` called "signal" setting its value to zero.
# Loop through the trading_df to create the signal column
for stock in stock_ticker_list:
    trading_df.loc[:,(stock, 'signal')] = 0.0

# Then we sort the columns to show within each stock
trading_df = trading_df.sort_index(axis=1, level=0)

# Review the trading_df
# Display the first and last five rows
display(trading_df.head())
display(trading_df.tail())

In [ ]:
# Set signal values
for stock in stock_ticker_list:
    # Create the signal to buy
    trading_df.loc[(trading_df[stock]['daily return'] >= 0), (stock, 'signal')] = 1
    # Create the signal to sell
    trading_df.loc[(trading_df[stock]['daily return'] < 0), (stock, 'signal')] = -1

# Review the trading_df
# Display the first and last five rows
display(trading_df.head())
display(trading_df.tail())

In [ ]:
# Copy the new "signal" column to a new Series called `y`.

# Create y DataFrame
y = pd.DataFrame(columns = trading_df.columns)

# Re-define stock_ticker_list to ensure integrity
stock_ticker_list = ["AMAT", "AMD", "BHP", "DDD", "F", "FCX", "INTC", "MSFT", "NVDA", "QCOM", "RIO", "TSLA", "TSM", "VALE"]

# Assign a copy of the `signal` column to the new DataFrame called `y`
# Loop through the trading_df
for stock in stock_ticker_list:
    y.loc[:,(stock, 'signal')] = trading_df[(stock, 'signal')].copy()

# Drop columns other than signal
y = y.drop(['close', 'daily return', 'lagged volume', 'sma_fast', 'sma_slow', 'stock volatility', 'volume'], axis='columns', level=1)

# Display the first and last five rows of the y DataFrame
display(y.head())
display(y.tail())

In [ ]:
# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)


In [ ]:
# Select the ending period for the training data with an offset of 1 month
training_end = X.index.min() + DateOffset(months=1)

# Display the training end date
print(training_end)

In [ ]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Display sample data for X_train
display(X_train.head(2))
display(X_train.tail(2))

# Display sample data for y_train
display(y_train.head(2))
display(y_train.tail(2))

In [ ]:
#Generate the Testing End
testing_end = training_end + DateOffset(months=1)

# Display the testing_end
print(testing_end)

In [ ]:
# Generate the X_test and y_test DataFrames
testing_end = training_end + DateOffset(months=1)
X_test = X.loc[training_end:testing_end]
y_test = y.loc[training_end:testing_end]

# Display sample data for X_test
display(X_test.head(2))
display(X_test.tail(2))

# Display sample data for y_test
display(y_test.head(2))
display(y_test.tail(2))


In [ ]:
# Stocks in Data Set are "AMAT", "AMD", "BHP", "DDD", "F", "FCX", "INTC", "MSFT", "NVDA", "QCOM", "RIO", "TSLA", "TSM", "VALE"

# Create a StandardScaler instance
# scaler = StandardScaler()

# Apply the scaler model to fit the X_train data
# X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)


In [ ]:
# Create the classifier model
# svm_model = svm.SVC()

# Fit the model to the data using X_train_scaled and y_train
# svm_model = svm_model.fit(X_train_scaled, y_train)

# Use the trained model to predict the trading signals for the training data
# training_signal_predictions = svm_model.predict(X_train_scaled)

# Display the sample predictions
# training_signal_predictions

In [ ]:
# Evaluate the model using a classification report
# training_report = classification_report(y_train, training_signal_predictions)
# print(training_report)

In [ ]:
# Stocks in Data Set are "AMAT", "AMD", "BHP", "DDD", "F", "FCX", "INTC", "MSFT", "NVDA", "QCOM", "RIO", "TSLA", "TSM", "VALE"

# Create StandardScaler instances
scaler_AMAT = StandardScaler()
scaler_AMD = StandardScaler()
scaler_BHP = StandardScaler()
scaler_DDD = StandardScaler()
scaler_F = StandardScaler()
scaler_FCX = StandardScaler()
scaler_INTC = StandardScaler()
scaler_MSFT = StandardScaler()
scaler_NVDA = StandardScaler()
scaler_QCOM = StandardScaler()
scaler_RIO = StandardScaler()
scaler_TSLA = StandardScaler()
scaler_TSM = StandardScaler()
scaler_VALE = StandardScaler()

# Apply the scaler model to fit the X_train data
X_scaler_AMAT = scaler_AMAT.fit(X_train['AMAT'])
X_scaler_AMD = scaler_AMD.fit(X_train['AMD'])
X_scaler_BHP = scaler_BHP.fit(X_train['BHP'])
X_scaler_DDD = scaler_DDD.fit(X_train['DDD'])
X_scaler_F = scaler_F.fit(X_train['F'])
X_scaler_FCX = scaler_FCX.fit(X_train['FCX'])
X_scaler_INTC = scaler_INTC.fit(X_train['INTC'])
X_scaler_MSFT = scaler_MSFT.fit(X_train['MSFT'])
X_scaler_NVDA = scaler_NVDA.fit(X_train['NVDA'])
X_scaler_QCOM = scaler_QCOM.fit(X_train['QCOM'])
X_scaler_RIO = scaler_RIO.fit(X_train['RIO'])
X_scaler_TSLA = scaler_TSLA.fit(X_train['TSLA'])
X_scaler_TSM = scaler_TSM.fit(X_train['TSM'])
X_scaler_VALE = scaler_VALE.fit(X_train['VALE'])

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_AMAT_scaled = X_scaler_AMAT.transform(X_train['AMAT'])
X_test_AMAT_scaled = X_scaler_AMAT.transform(X_test['AMAT'])

X_train_AMD_scaled = X_scaler_AMD.transform(X_train['AMD'])
X_test_AMD_scaled = X_scaler_AMD.transform(X_test['AMD'])

X_train_BHP_scaled = X_scaler_BHP.transform(X_train['BHP'])
X_test_BHP_scaled = X_scaler_BHP.transform(X_test['BHP'])

X_train_DDD_scaled = X_scaler_DDD.transform(X_train['DDD'])
X_test_DDD_scaled = X_scaler_DDD.transform(X_test['DDD'])

X_train_F_scaled = X_scaler_F.transform(X_train['F'])
X_test_F_scaled = X_scaler_F.transform(X_test['F'])

X_train_FCX_scaled = X_scaler_FCX.transform(X_train['FCX'])
X_test_FCX_scaled = X_scaler_FCX.transform(X_test['FCX'])

X_train_INTC_scaled = X_scaler_INTC.transform(X_train['INTC'])
X_test_INTC_scaled = X_scaler_INTC.transform(X_test['INTC'])

X_train_MSFT_scaled = X_scaler_MSFT.transform(X_train['MSFT'])
X_test_MSFT_scaled = X_scaler_MSFT.transform(X_test['MSFT'])

X_train_NVDA_scaled = X_scaler_NVDA.transform(X_train['NVDA'])
X_test_NVDA_scaled = X_scaler_NVDA.transform(X_test['NVDA'])

X_train_QCOM_scaled = X_scaler_QCOM.transform(X_train['QCOM'])
X_test_QCOM_scaled = X_scaler_QCOM.transform(X_test['QCOM'])

X_train_RIO_scaled = X_scaler_RIO.transform(X_train['RIO'])
X_test_RIO_scaled = X_scaler_RIO.transform(X_test['RIO'])

X_train_TSLA_scaled = X_scaler_TSLA.transform(X_train['TSLA'])
X_test_TSLA_scaled = X_scaler_TSLA.transform(X_test['TSLA'])

X_train_TSM_scaled = X_scaler_TSM.transform(X_train['TSM'])
X_test_TSM_scaled = X_scaler_TSM.transform(X_test['TSM'])

X_train_VALE_scaled = X_scaler_VALE.transform(X_train['VALE'])
X_test_VALE_scaled = X_scaler_VALE.transform(X_test['VALE'])


In [ ]:
# Stocks in Data Set are "AMAT", "AMD", "BHP", "DDD", "F", "FCX", "INTC", "MSFT", "NVDA", "QCOM", "RIO", "TSLA", "TSM", "VALE"

# Create the classifier model
svm_model_1 = svm.SVC()
svm_model_2 = svm.SVC()
svm_model_3 = svm.SVC()
svm_model_4 = svm.SVC()
svm_model_5 = svm.SVC()
svm_model_6 = svm.SVC()
svm_model_7 = svm.SVC()
svm_model_8 = svm.SVC()
svm_model_9 = svm.SVC()
svm_model_10 = svm.SVC()
svm_model_11 = svm.SVC()
svm_model_12 = svm.SVC()
svm_model_13 = svm.SVC()
svm_model_14 = svm.SVC()

# Fit the model to the data using X_train_scaled and y_train
svm_model_AMAT = svm_model_1.fit(X_train_AMAT_scaled, y_train['AMAT'])
svm_model_AMD = svm_model_2.fit(X_train_AMD_scaled, y_train['AMD'])
svm_model_BHP = svm_model_3.fit(X_train_BHP_scaled, y_train['BHP'])
svm_model_DDD = svm_model_4.fit(X_train_DDD_scaled, y_train['DDD'])
svm_model_F = svm_model_5.fit(X_train_F_scaled, y_train['F'])
svm_model_FCX = svm_model_6.fit(X_train_FCX_scaled, y_train['FCX'])
svm_model_INTC = svm_model_7.fit(X_train_INTC_scaled, y_train['INTC'])
svm_model_MSFT = svm_model_8.fit(X_train_MSFT_scaled, y_train['MSFT'])
svm_model_NVDA = svm_model_9.fit(X_train_NVDA_scaled, y_train['NVDA'])
svm_model_QCOM = svm_model_10.fit(X_train_QCOM_scaled, y_train['QCOM'])
svm_model_RIO = svm_model_11.fit(X_train_RIO_scaled, y_train['RIO'])
svm_model_TSLA = svm_model_12.fit(X_train_TSLA_scaled, y_train['TSLA'])
svm_model_TSM = svm_model_13.fit(X_train_TSM_scaled, y_train['TSM'])
svm_model_VALE = svm_model_14.fit(X_train_VALE_scaled, y_train['VALE'])

# Use the trained model to predict the trading signals for the training data
training_signal_predictions_AMAT = svm_model_AMAT.predict(X_train_AMAT_scaled)
training_signal_predictions_AMD = svm_model_AMD.predict(X_train_AMD_scaled)
training_signal_predictions_BHP = svm_model_BHP.predict(X_train_BHP_scaled)
training_signal_predictions_DDD = svm_model_DDD.predict(X_train_DDD_scaled)
training_signal_predictions_F = svm_model_F.predict(X_train_F_scaled)
training_signal_predictions_FCX = svm_model_FCX.predict(X_train_FCX_scaled)
training_signal_predictions_INTC = svm_model_INTC.predict(X_train_INTC_scaled)
training_signal_predictions_MSFT = svm_model_MSFT.predict(X_train_MSFT_scaled)
training_signal_predictions_NVDA = svm_model_NVDA.predict(X_train_NVDA_scaled)
training_signal_predictions_QCOM = svm_model_QCOM.predict(X_train_QCOM_scaled)
training_signal_predictions_RIO = svm_model_RIO.predict(X_train_RIO_scaled)
training_signal_predictions_TSLA = svm_model_TSLA.predict(X_train_TSLA_scaled)
training_signal_predictions_TSM = svm_model_TSM.predict(X_train_TSM_scaled)
training_signal_predictions_VALE = svm_model_VALE.predict(X_train_VALE_scaled)

# Display the sample predictions for AMAT
training_signal_predictions_AMAT[:10]

In [ ]:
# Stocks in Data Set are "AMAT", "AMD", "BHP", "DDD", "F", "FCX", "INTC", "MSFT", "NVDA", "QCOM", "RIO", "TSLA", "TSM", "VALE"

# Evaluate the model using a classification report for each of the stocks
training_report_AMAT = classification_report(y_train['AMAT'], training_signal_predictions_AMAT)
training_report_AMD = classification_report(y_train['AMD'], training_signal_predictions_AMD)
training_report_BHP = classification_report(y_train['BHP'], training_signal_predictions_BHP)
training_report_DDD = classification_report(y_train['DDD'], training_signal_predictions_DDD)
training_report_F = classification_report(y_train['F'], training_signal_predictions_F)
training_report_FCX = classification_report(y_train['FCX'], training_signal_predictions_FCX)
training_report_INTC = classification_report(y_train['INTC'], training_signal_predictions_INTC)
training_report_MSFT = classification_report(y_train['MSFT'], training_signal_predictions_MSFT)
training_report_NVDA = classification_report(y_train['NVDA'], training_signal_predictions_NVDA)
training_report_QCOM = classification_report(y_train['QCOM'], training_signal_predictions_QCOM)
training_report_RIO = classification_report(y_train['RIO'], training_signal_predictions_RIO)
training_report_TSLA = classification_report(y_train['TSLA'], training_signal_predictions_TSLA)
training_report_TSM = classification_report(y_train['TSM'], training_signal_predictions_TSM)
training_report_VALE = classification_report(y_train['VALE'], training_signal_predictions_VALE)

# Display the Classification Reports

print("----------------------------")

print("AMAT Training Report")
print(training_report_AMAT)

print("----------------------------")

print("AMD Training Report")
print(training_report_AMD)

print("----------------------------")

print("BHP Training Report")
print(training_report_BHP)

print("----------------------------")

print("DDD Training Report")
print(training_report_DDD)

print("----------------------------")

print("F Training Report")
print(training_report_F)

print("----------------------------")

print("FCX Training Report")
print(training_report_FCX)

print("----------------------------")

print("INTC Training Report")
print(training_report_INTC)

print("----------------------------")

print("MSFT Training Report")
print(training_report_MSFT)

print("----------------------------")

print("NVDA Training Report")
print(training_report_NVDA)

print("----------------------------")

print("QCOM Training Report")
print(training_report_QCOM)

print("----------------------------")

print("RIO Training Report")
print(training_report_RIO)

print("----------------------------")

print("TSLA Training Report")
print(training_report_TSLA)

print("----------------------------")

print("TSM Training Report")
print(training_report_TSM)

print("----------------------------")

print("VALE Training Report")
print(training_report_VALE)

print("----------------------------")


## Backtesting

In [ ]:
# Stocks in Data Set are "AMAT", "AMD", "BHP", "DDD", "F", "FCX", "INTC", "MSFT", "NVDA", "QCOM", "RIO", "TSLA", "TSM", "VALE"

# Use the trained models to predict the trading signals for the testing data.
testing_signal_predictions_AMAT = svm_model_AMAT.predict(X_test_AMAT_scaled)
testing_signal_predictions_AMD = svm_model_AMD.predict(X_test_AMD_scaled)
testing_signal_predictions_BHP = svm_model_BHP.predict(X_test_BHP_scaled)
testing_signal_predictions_DDD = svm_model_DDD.predict(X_test_DDD_scaled)
testing_signal_predictions_F = svm_model_F.predict(X_test_F_scaled)
testing_signal_predictions_FCX = svm_model_FCX.predict(X_test_FCX_scaled)
testing_signal_predictions_INTC = svm_model_INTC.predict(X_test_INTC_scaled)
testing_signal_predictions_MSFT = svm_model_MSFT.predict(X_test_MSFT_scaled)
testing_signal_predictions_NVDA = svm_model_NVDA.predict(X_test_NVDA_scaled)
testing_signal_predictions_QCOM = svm_model_QCOM.predict(X_test_QCOM_scaled)
testing_signal_predictions_RIO = svm_model_RIO.predict(X_test_RIO_scaled)
testing_signal_predictions_TSLA = svm_model_TSLA.predict(X_test_TSLA_scaled)
testing_signal_predictions_TSM = svm_model_TSM.predict(X_test_TSM_scaled)
testing_signal_predictions_VALE = svm_model_VALE.predict(X_test_VALE_scaled)

# Evaluate the model's ability to predict the trading signal for the testing data
testing_report_AMAT = classification_report(y_test['AMAT'], testing_signal_predictions_AMAT)
testing_report_AMD = classification_report(y_test['AMD'], testing_signal_predictions_AMD)
testing_report_BHP = classification_report(y_test['BHP'], testing_signal_predictions_BHP)
testing_report_DDD = classification_report(y_test['DDD'], testing_signal_predictions_DDD)
testing_report_F = classification_report(y_test['F'], testing_signal_predictions_F)
testing_report_FCX = classification_report(y_test['FCX'], testing_signal_predictions_FCX)
testing_report_INTC = classification_report(y_test['INTC'], testing_signal_predictions_INTC)
testing_report_MSFT = classification_report(y_test['MSFT'], testing_signal_predictions_MSFT)
testing_report_NVDA = classification_report(y_test['NVDA'], testing_signal_predictions_NVDA)
testing_report_QCOM = classification_report(y_test['QCOM'], testing_signal_predictions_QCOM)
testing_report_RIO = classification_report(y_test['RIO'], testing_signal_predictions_RIO)
testing_report_TSLA = classification_report(y_test['TSLA'], testing_signal_predictions_TSLA)
testing_report_TSM = classification_report(y_test['TSM'], testing_signal_predictions_TSM)
testing_report_VALE = classification_report(y_test['VALE'], testing_signal_predictions_VALE)

# Display the Classification Reports

print("----------------------------")

print("AMAT Testing Report")
print(testing_report_AMAT)

print("----------------------------")

print("AMD Testing Report")
print(testing_report_AMD)

print("----------------------------")

print("BHP Testing Report")
print(testing_report_BHP)

print("----------------------------")

print("DDD Testing Report")
print(testing_report_DDD)

print("----------------------------")

print("F Testing Report")
print(testing_report_F)

print("----------------------------")

print("FCX Testing Report")
print(testing_report_FCX)

print("----------------------------")

print("INTC Testing Report")
print(testing_report_INTC)

print("----------------------------")

print("MSFT Testing Report")
print(testing_report_MSFT)

print("----------------------------")

print("NVDA Testing Report")
print(testing_report_NVDA)

print("----------------------------")

print("QCOM Testing Report")
print(testing_report_QCOM)

print("----------------------------")

print("RIO Testing Report")
print(testing_report_RIO)

print("----------------------------")

print("TSLA Testing Report")
print(testing_report_TSLA)

print("----------------------------")

print("TSM Testing Report")
print(testing_report_TSM)

print("----------------------------")

print("VALE Testing Report")
print(testing_report_VALE)

print("----------------------------")
